In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import root, fsolve
import pandas as pd

In [2]:
# Initial data
T_idle = 3
T_wait = 33
T_data = 30
T_rts = 3
T_cts = 3
T_ack = 3
T_bo = 3
T_out = 33

T_max = 12
Nr = 3
p_a = 1
nodes = 10

### Formulas for usual backoff

In [3]:
# Cycle time
def Etc(p, p_f):
    if p_f == 1.:
        p_f = 0.9999999
    elif p_f == 0.:
        p_f = 0.0000001
    total_sum = 0
    for i in range(1, Nr + 1 + 1):
        first_addition = T_wait + T_bo / (1 - p_f)
        numerator = (i * T_max + 1) * (p_f ** (i * T_max + 1) - i * T_max * p_f - p_f + i * T_max)
        denominator = i * T_max * (1 - p_f ** (i * T_max + 1))
        second_addition = (1 - p) * (T_cts + T_data + T_ack)
        huge_sum = T_rts + p * T_out / p_f + first_addition * numerator / denominator + second_addition
        total_sum += p ** (i - 1) * huge_sum
    return T_idle + p_a * total_sum

In [4]:
def pi_idle(p, p_f):
    return T_idle / Etc(p, p_f)

def pi_rts(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = p ** (i - 1) * p_a * T_rts
        denominator = Etc(p, p_f)
        total_sum += numerator / denominator
    return total_sum

def pi_out(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = p ** (i - 1) * p_a * p * T_out
        denominator = Etc(p, p_f) * p_f
        total_sum += numerator / denominator
    return total_sum

def pi_cts(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = (1 - p) * p ** (i - 1) * p_a * T_cts
        denominator = Etc(p, p_f)
        total_sum += numerator / denominator
    return total_sum

def pi_bo(p, p_f):
    if p_f == 1.:
        p_f = 0.9999999
    elif p_f == 0.:
        p_f = 0.0000001
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = p ** (i - 1) * p_a * T_bo * (i * T_max + 1) * (p_f ** (i * T_max + 1) - i * T_max * p_f - p_f + i * T_max)
        denominator = Etc(p, p_f) * (i * T_max) * (1 - p_f) * (1 - p_f ** (i * T_max + 1))
        total_sum += numerator / denominator
    return total_sum

def pi_wait(p, p_f):
    if p_f == 1.:
        p_f = 0.9999999
    elif p_f == 0.:
        p_f = 0.0000001
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = p ** (i - 1) * p_a * T_wait * (i * T_max + 1) * (p_f ** (i * T_max + 1) - i * T_max * p_f - p_f + i * T_max)
        denominator = Etc(p, p_f) * (i * T_max) * (1 - p_f ** (i * T_max + 1))
        total_sum += numerator / denominator
    return total_sum

def pi_data(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = (1 - p) * p ** (i - 1) * p_a * T_data
        denominator = Etc(p, p_f)
        total_sum += numerator / denominator
    return total_sum

def pi_ack(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = (1 - p) * p ** (i - 1) * p_a * T_ack
        denominator = Etc(p, p_f)
        total_sum += numerator / denominator
    return total_sum

In [5]:
def p_success(p):
    total_sum = 0
    for i in range(1, Nr + 1 + 1):
        total_sum += (1 - p) * p ** (i - 1) * p_a
    return total_sum

# alternative formula
def p_success(p):
    return 1 - p ** (Nr + 1)

def p_failure(p):
    return p ** (Nr + 1)

In [6]:
def transcendental_equation_p_collision(p, p_f, n):
    p_rts = pi_rts(p, p_f)
    p_cts = pi_cts(p, p_f)
    p_data = pi_data(p, p_f)
    p_ack = pi_ack(p, p_f)
    # We solve system for transcendental equation f(x) = g(x) by turning it into f(x) - g(x) = 0
    return p - (1 - (1 - p_rts) ** (2*(n-1))) - (1 - (1 - p_cts) ** (2*(n-1)))

def transcendental_equation_p_free(p, p_f, n):
    p_rts = pi_rts(p, p_f)
    p_cts = pi_cts(p, p_f)
    p_data = pi_data(p, p_f)
    p_ack = pi_ack(p, p_f)
    return p_f - (1 - p_cts) ** (n-1)

def system_of_equations(p_pf, node):
    p, pf = p_pf
    return (transcendental_equation_p_collision(p, pf, node), transcendental_equation_p_free(p, pf, node))

def calculate_p_pf_from_system():
    p_array = []
    pf_array = []
    for node in range(1, nodes+1):
        p, pf = fsolve(system_of_equations, (0.01, 0.01), args=(node))
        p_array.append(p)
        pf_array.append(pf)
    return p_array, pf_array

In [7]:
p_array, p_f_array = calculate_p_pf_from_system()

In [8]:
p_array

[1.8086627663754192e-27,
 0.14487822697094868,
 0.23279380562357307,
 0.29622715083755957,
 0.34635817002693897,
 0.38817689322821136,
 0.4243079559426458,
 0.45629512208391565,
 0.48512215365912814,
 0.5114546593007134]

In [9]:
p_f_array

[1.0,
 0.9659774331360108,
 0.9477568906451805,
 0.9359454425676955,
 0.9275006543720409,
 0.9211173689404055,
 0.9161286562949312,
 0.9121511770390431,
 0.9089454278476604,
 0.9063522598377893]

In [10]:
analytics_headers = [
    'nodes',
    'p_collision',
    'p_failure',
    'p_success',
    'p_free',
    'cycle_time',
    'idle_time',
    'bo_time',
    'rts_time',
    'cts_time',
    'data_time',
    'ack_time',
    'out_time',
    'wait_time',
]

In [11]:
data = [analytics_headers]
for i in range(nodes):
    data.append([
        i + 1,
        p_array[i],
        p_failure(p_array[i]),
        p_success(p_array[i]),
        p_f_array[i],
        Etc(p_array[i], p_f_array[i]),
        pi_idle(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_bo(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_rts(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_cts(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_data(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_ack(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_out(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_wait(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
    ])

In [12]:
# Draw table to evaluate the results
data_pd = {}
for i in range(1, len(data[0])):
    data_pd[data[0][i]] = np.array(data[1:]).T[i]
df1 = pd.DataFrame(data_pd, index=np.arange(1, len(np.array(data[1:]).T[0])+1))
df1

,p_collision,p_failure,p_success,p_free,cycle_time,idle_time,bo_time,rts_time,cts_time,data_time,ack_time,out_time,wait_time
1,1.808663e-27,1.070115e-107,1.000000,1.000000,61.495549,3.0,19.495528,3.000000,3.000000,30.000000,3.000000,5.968587e-26,0.000021
2,1.448782e-01,4.405675e-04,0.999559,0.965977,88.137921,3.0,29.006179,3.506727,2.998678,29.986783,2.998678,5.785365e+00,10.855511
3,2.327938e-01,2.936876e-03,0.997063,0.947757,112.994732,3.0,37.891975,3.898808,2.991189,29.911894,2.991189,1.053414e+01,21.775540
4,2.962272e-01,7.700155e-03,0.992300,0.935945,136.402625,3.0,46.182939,4.229915,2.976900,29.768995,2.976900,1.472647e+01,32.540505
5,3.463582e-01,1.439136e-02,0.985609,0.927501,158.631922,3.0,53.988815,4.523618,2.956826,29.568259,2.956826,1.858189e+01,43.055691
6,3.881769e-01,2.270485e-02,0.977295,0.921117,179.878344,3.0,61.406560,4.792048,2.931885,29.318854,2.931885,2.221409e+01,53.283022
7,4.243080e-01,3.241341e-02,0.967587,0.916129,200.277547,3.0,68.508682,5.042209,2.902760,29.027598,2.902760,2.568847e+01,63.205067
8,4.562951e-01,4.334942e-02,0.956651,0.912151,219.923849,3.0,75.348291,5.278510,2.869952,28.699517,2.869952,2.904578e+01,72.811846
9,4.851222e-01,5.538656e-02,0.944613,0.908945,238.883546,3.0,81.964698,5.503908,2.833840,28.338403,2.833840,3.231299e+01,82.095866
10,5.114547e-01,6.842717e-02,0.931573,0.906352,257.203548,3.0,88.387476,5.720490,2.794719,27.947185,2.794719,3.550880e+01,91.050161


In [13]:
# Here we save our table to csv in the same folder
with open('2021-03-10 sensing in OUT_usual_backoff.csv', "wt", newline="") as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(analytics_headers)
    for i in range(1, len(data)):
        writer.writerow(data[i])